### Module Importation and DataFrame Setup

In [1]:
# Reset the model for re-run
from tensorflow.keras import backend as K
K.clear_session()

# Import dependencies
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Read the CSV file from the Dataset folder into a Pandas DataFrame
dfSurvivals = pd.read_csv(
    Path("../CSV_OUTPUT/Base_Cleaned_DS_CSV.csv")
)

In [3]:
# Review the DataFrame
dfSurvivals.head()

,Passenger Id,Title,First Name,Last Name,Sex,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Embarkation Port,Survival Boat,Body Number,Survived,age_group,family_size
0,1,Miss,Elisabeth Walton,Allen,female,29.0000,0,0,1,211.3375,S,2,S,1,Adults,0
1,2,Master,Hudson Trevor,Allison,male,0.9167,1,2,1,151.5500,S,11,S,1,Children,3
2,3,Miss,Helen Loraine,Allison,female,2.0000,1,2,1,151.5500,S,DNS,BNR,0,Children,3
3,4,Mr,Hudson Joshua Creighton,Allison,male,30.0000,1,2,1,151.5500,S,DNS,135,0,Adults,3
4,5,Mrs,Hudson J C (Bessie Waldo Daniels),Allison,female,25.0000,1,2,1,151.5500,S,DNS,BNR,0,Adults,3


In [4]:
# Print DataFrame information to ensure no null values and check datatypes
dfSurvivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Passenger Id            1308 non-null   int64  
 1   Title                   1308 non-null   object 
 2   First Name              1308 non-null   object 
 3   Last Name               1308 non-null   object 
 4   Sex                     1308 non-null   object 
 5   Age                     1308 non-null   float64
 6   Sibling/Spouse Aboard   1308 non-null   int64  
 7   Parent/Children Aboard  1308 non-null   int64  
 8   Passenger Class         1308 non-null   int64  
 9   Fare                    1308 non-null   float64
 10  Embarkation Port        1308 non-null   object 
 11  Survival Boat           1308 non-null   object 
 12  Body Number             1308 non-null   object 
 13  Survived                1308 non-null   int64  
 14  age_group               1308 non-null   

In [5]:
# Preprocessing for features which are currently objects
labelencoder_X = LabelEncoder()
dfSurvivals["Title"] = labelencoder_X.fit_transform(dfSurvivals["Title"])
dfSurvivals["Last Name"] = labelencoder_X.fit_transform(dfSurvivals["Last Name"])
dfSurvivals["Sex"] = labelencoder_X.fit_transform(dfSurvivals["Sex"])
dfSurvivals["Embarkation Port"] = labelencoder_X.fit_transform(dfSurvivals["Embarkation Port"])
dfSurvivals["age_group"] = labelencoder_X.fit_transform(dfSurvivals["age_group"])

In [6]:
# Drop columns
dfSurvivals = dfSurvivals.drop(['Passenger Id', 'First Name', 'Survival Boat', 'Body Number'], axis=1)
dfSurvivals.head()

,Title,Last Name,Sex,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Embarkation Port,Survived,age_group,family_size
0,9,15,0,29.0000,0,0,1,211.3375,2,1,0,0
1,8,16,1,0.9167,1,2,1,151.5500,2,1,1,3
2,9,16,0,2.0000,1,2,1,151.5500,2,0,1,3
3,12,16,1,30.0000,1,2,1,151.5500,2,0,0,3
4,13,16,0,25.0000,1,2,1,151.5500,2,0,0,3


In [7]:
# Ensure all columns are numerical
dfSurvivals.dtypes

Title                       int32
Last Name                   int32
Sex                         int32
Age                       float64
Sibling/Spouse Aboard       int64
Parent/Children Aboard      int64
Passenger Class             int64
Fare                      float64
Embarkation Port            int32
Survived                    int64
age_group                   int32
family_size                 int64
dtype: object

### Oversample to Correct Imbalance in Dataset
#### Oversampling code example used from https://www.section.io/engineering-education/how-to-handle-imbalanced-data-in-deep-neural-networks/#building-the-deep-neural-network-model-using-an-imbalanced-dataset

In [11]:
# Count the number of data samples in each class
count_majority_class, count_minority_class = dfSurvivals.Survived.value_counts()

In [12]:
# Save majority and minority classes in new variables
# In Survived column 0=No and 1=Yes 

df_majority_class = dfSurvivals[dfSurvivals['Survived'] == 0]
df_minority_class = dfSurvivals[dfSurvivals['Survived'] == 1]

In [13]:
# Oversample the minority data
df_class_oversample = df_minority_class.sample(count_majority_class, replace=True)

In [15]:
# Concatenate the oversampled minority class with the majority class
df_balanced_os = pd.concat([df_class_oversample, df_majority_class], axis=0)

In [17]:
# Count number of data samples in new balanced dataset
print('Number of data samples after over-sampling:')
print(df_balanced_os.Survived.value_counts())

Number of data samples after over-sampling:
1    808
0    808
Name: Survived, dtype: int64


In [19]:
# Review the DataFrame
df_balanced_os.head()

,Title,Last Name,Sex,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Embarkation Port,Survived,age_group,family_size
358,13,114,0,42.0,0,0,2,13.0000,2,1,0,0
735,13,175,0,36.0,0,2,3,15.9000,2,1,0,2
195,9,480,0,16.0,0,0,1,86.5000,2,1,3,0
1037,13,544,0,31.0,0,0,3,7.2292,0,1,0,0
95,13,211,0,54.0,1,1,1,81.8583,2,1,0,2


### Split the data into X and y and then into testing and training sets

In [21]:
# Split the data into X (features) and y (target)

# Set the y variable, which is the target
y = df_balanced_os['Survived']

# Set the X variable, which includes all features escept the target
X = df_balanced_os.drop(columns=['Survived'])

In [22]:
# Split into testing and training sets using train_test_split
# Stratify argument ensures that the classes remain balanced after splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train, and Evaluate the Model

In [24]:
# Get the number of input features
input_dim = X_train.shape[1]

# Function to create and compile the neural network
def create_model(hp):
    
    # Create a new model
    nn_model = tf.keras.models.Sequential()
    
    # Choose activation function for hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Add first hidden layer with hyperparameter-tuned settings
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units', 
        min_value=1, 
        max_value=10, 
        step=2), 
        activation=activation, 
        input_dim=input_dim))
    
    # Add second hidden layer with 10 neurons
    nn_model.add(tf.keras.layers.Dense(units=10, activation=activation))
    
    # Add output layer for binary classification
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy", "Precision", "Recall"]) 
    
    # Return compiled model
    return nn_model  

In [25]:
# Set up the kerastuner then allow it to search for best hyperparameters
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

# Perform hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 21 Complete [00h 00m 02s]
val_accuracy: 0.7029703259468079

Best val_accuracy So Far: 0.7920792102813721
Total elapsed time: 00h 00m 43s
INFO:tensorflow:Oracle triggered exit


In [26]:
# Get the hyperparameters of the best model
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0015'}

In [27]:
# Get the best model
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 36        
                                                                 
 dense_1 (Dense)             (None, 10)                40        
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 87 (348.00 Byte)
Trainable params: 87 (348.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Train the model
fit_model = best_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
38/38 [==============================] - 1s 1ms/step - loss: 0.4558 - accuracy: 0.8020 - precision: 0.8315 - recall: 0.7574
Epoch 2/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4549 - accuracy: 0.8036 - precision: 0.8321 - recall: 0.7607
Epoch 3/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4547 - accuracy: 0.8061 - precision: 0.8354 - recall: 0.7624
Epoch 4/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4534 - accuracy: 0.8061 - precision: 0.8391 - recall: 0.7574
Epoch 5/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4528 - accuracy: 0.8069 - precision: 0.8382 - recall: 0.7607
Epoch 6/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4522 - accuracy: 0.8069 - precision: 0.8370 - recall: 0.7624
Epoch 7/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4517 - accuracy: 0.8086 - precision: 0.8388 - recall: 0.7640
Epoch 8/100
38/38 [=============================

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, Precision: {model_precision}, Recall: {model_recall}")

13/13 - 0s - loss: 0.4673 - accuracy: 0.8020 - precision: 0.8389 - recall: 0.7475 - 228ms/epoch - 18ms/step
Loss: 0.4673449695110321, Accuracy: 0.801980197429657, Precision: 0.8388888835906982, Recall: 0.7475247383117676


In [30]:
# Use the model to make predictions on test data
y_pred_prob = best_model.predict(X_test_scaled).flatten()
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

13/13 [==============================] - 0s 834us/step
Confusion Matrix:
[[173  29]
 [ 51 151]]
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.86      0.81       202
           1       0.84      0.75      0.79       202

    accuracy                           0.80       404
   macro avg       0.81      0.80      0.80       404
weighted avg       0.81      0.80      0.80       404



In [31]:
# Export our model to keras file
best_model.save('OUTPUT/Titanic_survival_mod1_reprocessed_opt1.keras')